# King County Millenial homeowner program

**Buying a property in King County recommendations for Millennials**

**Authors:** Hanis Zulmuthi, Greg Burgess, Kyle Weesner

***March 2022***


<img src="https://www.condosky.com/wp-content/uploads/2019/06/Young-Families-Condos-Toronto-e1561577724820.jpg" alt="A picture of a multiracial family sitting on the floor together surrounded by moving boxes" title="" />

# Overview

King County in Washington State is growing in population made up of millenials and generation Z who either grew up or migrated to the area and found life in the region. We, the Data Analysts at the Greek Honey real estate are woking on building a model that predicts house prices in King county with the goal to help folks from the demographic to find their dream house that fits their lifestyle and their pockets! 

# ADD A SUMMARY OF RESULTS HERE

## Business Problem

Housing market in King County region, especially in the major cities such as Seattle, are becoming more and more financially inaccessible for many demographics. The demographic of interest for our real-estate is millenials who are building their career, family and trying to break into the housing market to have a stable housing for themselves. With our house price predicition model, we will help them to find the market value of their ideal house that are in proximity to city centers.

## Data Understanding

### 1. King County Housing data ([kc_house_data.csv](https://github.com/hanis-z/Phase-2-project/blob/main/data/kc_house_data.csv))
**Source:** This data was provided to us by Flatiron School. This data file is available in the project repo in the folder "data". 

**Contents:**

### 2. Incorporated & Unincorporated cities in King county
**Source:** We downloaded this data from [King County GIS website]( https://gis-kingcounty.opendata.arcgis.com/datasets/kingcounty::cities-and-unincorporated-king-county-city-kc-area/explore?location=47.430582%2C-121.809200%2C10.02). The data is also easily accessible in our project repo in the folder "data".

**Contents:**

### 3. Neighborhood Map of Seattle
**Source:** We attained this data from [Seattle city GIS website](https://data-seattlecitygis.opendata.arcgis.com/datasets/neighborhood-map-atlas-districts/explore?location=47.628714%2C-122.338313%2C11.43]). This data is also easily accessible in our project repo in the folder "data".

**Contents:**


### Importing python libraries required for the analysis.

**Note:** To ensure that all packages are loaded successfully, please ensure that you are in the right python environment. For more reference, refer the (how-to-set-environment)[link to the file] to ensure you have the correct working environment to run this notebook.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')


from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import RFE
from pandas.api.types import is_numeric_dtype


### Define Helper Functions for analysis

In [2]:
def dataframe_info(df):
    '''
    Takes Pandas dataframe (df). Prints number of rows, number of columns, and three sample rows.
    Returns list of numerical columns and list of non-numerical columns.
    '''
    numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
    object_cols  = df.select_dtypes(exclude=np.number).columns.tolist()

    nrows, ncol = df.shape
    nmid = nrows // 2
    
    print(f"Dimensions: {nrows} rows and {ncol} columns")
    print(f"Numeric columns: {len(numeric_cols)}")
    print(f"Object columns: {len(object_cols)}")

    display(df.iloc[[0,nmid,nrows-1]]) # show three rows: first, mid, and last

    return numeric_cols, object_cols

## Data Exploration & Preparation for Linear Regression Modelling


### Load King County Housing data

In [3]:
housing_df = pd.read_csv('data/housing_gdf_complete.csv')

In [4]:
dataframe_info(housing_df)

Dimensions: 21596 rows and 28 columns
Numeric columns: 17
Object columns: 11


,Unnamed: 0,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,...,lat,long,sqft_living15,sqft_lot15,coord,geometry,city,dist_seattle,neighborhood,in_seattle
0,0,8856004730,9/17/2014,199950.0,2,2.75,1590,20917,1.5,NO,...,47.2786,-122.250,1310,6000,"(-122.25, 47.2786)",POINT (392815.9311460542 31962.040946318088),Algona,38586.298196,NaN,False
10798,10798,7237500590,11/17/2014,1320000.0,4,5.25,6110,10369,2.0,NO,...,47.5285,-122.135,4190,10762,"(-122.135, 47.5285)",POINT (401985.90108066204 59588.48465827319),Newcastle,19145.331588,NaN,False
21595,21595,9808100150,4/2/2015,3350000.0,5,3.75,5350,15360,1.0,NO,...,47.6480,-122.218,3740,15940,"(-122.218, 47.648)",POINT (395975.83394082234 72981.52286293103),Yarrow Point,10564.303190,NaN,False


(['Unnamed: 0',
  'id',
  'price',
  'bedrooms',
  'bathrooms',
  'sqft_living',
  'sqft_lot',
  'floors',
  'sqft_above',
  'yr_built',
  'yr_renovated',
  'zipcode',
  'lat',
  'long',
  'sqft_living15',
  'sqft_lot15',
  'dist_seattle'],
 ['date',
  'waterfront',
  'view',
  'condition',
  'grade',
  'sqft_basement',
  'coord',
  'geometry',
  'city',
  'neighborhood',
  'in_seattle'])

In [6]:
housing_df['dist_seattle'] = housing_df['dist_seattle']/1000 #converting the distance from meters to kilometers

## Data Cleaning

### Basement

Creating a feature if the house has a basement or not.

In [7]:
# sqft_basement should equal difference between sqft_living and sqft_above
sqft_basement_computed = housing_df['sqft_living'] - housing_df['sqft_above']

housing_df['has_basement'] = (sqft_basement_computed > 0).astype(int)

### Yr renovated

Creating a feature of houses that have been renovated in the past 5 years.

In [12]:
housing_df['sold_dt'] = pd.to_datetime(housing_df['date']) #made a new column with date by using the pd.to_datetime()
housing_df['sold_year'] = pd.DatetimeIndex(housing_df['sold_dt']).year #made a new column year sold 
housing_df['sold_month'] = pd.DatetimeIndex(housing_df['sold_dt']).month #made a new column month sold 

In [13]:
housing_df['yr_renovated_missing'] = (housing_df['yr_renovated'].isna()).astype(int) #saving nulls for ablilty to work with
housing_df['yr_renovated'].fillna(0, inplace=True) #Rewriting dataframe filling null values with zeros

housing_df['renovated'] = ((housing_df['sold_year'] - housing_df['yr_renovated']) <= 5).astype(int) 
#houses renovated within the past 5 years

### Age of the house

Creating a feature of the age in years of the houses.

In [14]:
housing_df['house_age'] = housing_df['sold_year'] - housing_df['yr_built'] #year sold minus year built gets the age of the house
housing_df['house_age'].describe()

count    21596.000000
mean        43.323810
std         29.377864
min         -1.000000
25%         18.000000
50%         40.000000
75%         63.000000
max        115.000000
Name: house_age, dtype: float64

### View

Cleaning up missing values in the view column before using it as a feature.

In [18]:
housing_df['view_missing'] = (housing_df['view'].isna()).astype(int) #saving nulls for ablilty to work with
housing_df['view'].fillna('NONE', inplace=True) #Rewriting dataframe filling null values with NONE

### Waterfront

Cleaning up missing values in the waterfront column before using it as a feature.

In [19]:
housing_df['waterfront_missing'] = (housing_df['waterfront'].isna()).astype(int)#saving nulls for ablilty to work with
housing_df['waterfront'].fillna('NO', inplace=True) #Rewriting dataframe filling null values with NO

### Duplicate records (house resold)

Creating a feature of individual houses that were sold more than once (stated as a boolean value).

In [20]:
print(f"Number of rows:\t\t{len(housing_df['id'])}") #Total number of houses sold on id including repeats
print(f"Number unique:\t\t{len(housing_df['id'].unique())}") #First time houses being sold in this data set
print(f"Number duplicates:\t{sum(housing_df['id'].duplicated() == True)}") #houses being resold

Number of rows:		21596
Number unique:		21419
Number duplicates:	177


In [23]:
housing_df = housing_df.sort_values(by=['sold_year', 'sold_month']) 
#sorting df by their dates so resold values are second in the dataframe
housing_df['resold'] = housing_df.id.duplicated().astype(int) #creates a boolean column when the id of houses are duplicated

### Ratio between bedrooms and bathrooms

Creating a feature of the ratio between bedrooms and bathrooms.

In [24]:
housing_df['br_bth'] = housing_df.bedrooms / housing_df.bathrooms

### In seattle

In [ ]:
housing_df['in_seattle'] = housing_df['in_seattle'].astype(int)

Eliminating condition because grade does a better job (Refer Greg's original notebook)

In [ ]:
relevant_num_cols = ['bedrooms',
  'sqft_living',
  'sqft_lot',
  'floors',
  'zipcode',
  'dist_seattle',
  'has_basement',
  'yr_renovated_missing',
  'house_age',
  'view_missing',
  'waterfront_missing','resold', 'br_bth']

relevant_cat_col =  ['waterfront',
  'view',
  'condition',
  'grade',
  'city',
  'neighborhood'
  ]

In [ ]:
sns.heatmap(housing_df[relevant_num_cols].corr().abs(), vmin=.7, vmax=0.7)

### Train test split

In [ ]:
y = housing_df['price']
X = housing_df[relevant_num_cols+relevant_cat_col]

In [ ]:
X.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


## One Hot Encoder

In [ ]:
dataframe_info(X_train)

In [ ]:
encoder = OneHotEncoder(handle_unknown='ignore',
                        drop='if_binary',
                        categories='auto')

ct = ColumnTransformer(transformers=[('ohe', encoder, relevant_cat_col)],
                       remainder='passthrough')

ct.fit(X_train)
X_train_enc = ct.transform(X_train)
X_test_enc = ct.transform(X_test)

drop: ohe__x1_NONE, ohe__x2_Average, 'ohe__x3_3 Poor', 'ohe_x3_4 Low', 'ohe_x3_5 Fair', 'ohe_x4_Unincorporated City', 'ohe__x5_nan'


In [ ]:
# can display as a dataframe like so
X_train_enc_df = pd.DataFrame(X_train_enc.todense(), columns= ct.get_feature_names(), index =X_train.index)
X_train_enc_df

In [ ]:
train_df = pd.concat([X_train,y_train], axis=1)

train_df.corr().abs()['price'][train_df.corr().abs()['price'].index == 'sqft_living']

In [ ]:
def build_model(X_train, X_test, y_train, y_test):
    
    """
    Build a regression model
    
    """
    
    lr = LinearRegression()
    lr.fit(X_train,y_train)
    R2_train = lr.score(X_train,y_train)
    R2_test = lr.score(X_test,y_test)


    yhat_train = lr.predict(X_train)
    train_rmse = mean_squared_error(y_train, yhat_train, squared = False)

    yhat_test = lr.predict(X_test)
    test_rmse = mean_squared_error(y_test, yhat_test, squared = False)
    
    print(f'Train R2: {baseline_lr.score(X1_train,y_train)}')
    print(f'Test R2: {baseline_lr.score(X1_test,y_test)}')
    print(f'Train RMSE: {train_rmse}')
    print(f'Test RMSE: {test_rmse}')
    
    return R2_train, R2_test, train_rmse, test_rmse


In [ ]:
#Build baseline model

X1_train = X_train[['sqft_living']]
X1_test = X_test[['sqft_living']]

baseline_lr = LinearRegression()
baseline_lr.fit(X1_train,y_train)

print(f'Baseline_train R2: {baseline_lr.score(X1_train,y_train)}')
print(f'Baseline_test R2: {baseline_lr.score(X1_test,y_test)}')

yhat_train = baseline_lr.predict(X1_train)
train_rmse = mean_squared_error(y_train, yhat_train, squared = False)
print(f'Train RMSE: {train_rmse}')

yhat_test = baseline_lr.predict(X1_test)
test_rmse = mean_squared_error(y_test, yhat_test, squared = False)
print(f'Test RMSE: {test_rmse}')



In [ ]:
X2_train = X_train[['sqft_living','bedrooms']]
X2_test = X_test[['sqft_living','bedrooms']]

model_1_r2_train, model_1_r2_test, train_rmse, test_rmse = build_model(X2_train,X2_test,y_train,y_test)


In [ ]:
ct.get_feature_names()

drop: ohe__x1_NONE, ohe__x2_Average, 'ohe__x3_3 Poor', 'ohe_x3_4 Low', 'ohe_x3_5 Fair', 'ohe_x4_Unincorporated City', 'ohe__x5_nan'

In [ ]:
ori_cols = ['ohe__x0_YES',
 'ohe__x1_AVERAGE',
 'ohe__x1_EXCELLENT',
 'ohe__x1_FAIR',
 'ohe__x1_GOOD',
 'ohe__x2_Average',
 'ohe__x2_Fair',
 'ohe__x2_Good',
 'ohe__x2_Poor',
 'ohe__x2_Very Good',
 'ohe__x3_10 Very Good',
 'ohe__x3_11 Excellent',
 'ohe__x3_12 Luxury',
 'ohe__x3_13 Mansion',
 'ohe__x3_5 Fair',
 'ohe__x3_6 Low Average',
 'ohe__x3_8 Good',
 'ohe__x3_9 Better',
 'bedrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'zipcode',
 'has_basement',
 'yr_renovated_missing',
 'house_age',
 'view_missing',
 'waterfront_missing',
 'resold',
 'br_bth']

## Data Visualization

In [ ]:

# # fig, ax = plt.subplots(ncols = 2,)
# f, ax = plt.subplots(1, 2)

g = sns.lmplot(x='sqft_living', y='price', hue = 'in_seattle',data=housing_df[housing_df['price']<2000000], scatter = False,facet_kws={'legend_out': True},height = 7)
# ax1.set_xlabel ('Living space (sqft)')
# ax1.set_ylabel('Price')


# plt.figure(figsize = (15,20))
plt.xlabel("Living space (sqft)")
plt.ylabel("Price ($)")
plt.title('In Seattle vs outside Seattle', y=1, fontsize=10)
plt.suptitle('Price by Living Space',y=1.04,fontsize = 18)
# title
new_title = 'In Seattle'
g._legend.set_title(new_title)
# replace labels
new_labels = ['No', 'Yes']
for t, l in zip(g._legend.texts, new_labels):
    t.set_text(l)
plt.ticklabel_format(style='plain', axis='y');


plt.savefig('Images/price_by_living_space.jpg',bbox_inches="tight",dpi=300)
plt.savefig('Images/price_by_living_space.png',bbox_inches="tight",dpi=300)

plt.show(g)

The figure for median house price by neighborhood in Seattle is plotted in the notebook ***add_city_neighborhood.ipynb*** , available in our repo. For any interest in geospatial processing done in this project, please refer to the following notebook.

<img src='Images/mean_price_by_neighborhood_map.png' alt="Figure of Median house price by neighborhood in Seattle" title="Median house price by neighborhood in Seattle" />

<img src='Images/property_price_map.jpg' alt="Figure of property price across King county, Washington" title="Property price across King county, Washington" />